# Interactive Recommendation System with Word Embeddings using Word2Vec, Plotly, and NetworkX

## Project Breakdown
- Task 1: Introduction
- Task 2: Exploratory Data Analysis and Preprocessing
- Task 3: Word2Vec with Gensim
- Task 4: Exploring Results
- ## Task 5: Building and Visualizing Interactive Network Graph
Let's see if we can visualize our results better!

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
from plotly.offline import plot
import networkx as nx
import numpy as np

In [3]:
words = np.loadtxt('Data/words.txt', dtype=str)
vectors = np.load('Data/vectors.npy')

In [4]:
sims = cosine_similarity(vectors, vectors)

In [5]:
for i in range(len(vectors)):
    for j in range(len(vectors)):
        if i<=j:
            sims[i,j]= False

In [7]:
indeces = np.argwhere(sims>0.65)
indeces

array([[ 18,  16],
       [ 30,   9],
       [ 31,   9],
       ...,
       [907, 847],
       [907, 886],
       [909, 388]])

In [8]:
G = nx.Graph()

for index in indeces:
    G.add_edge(words[index[0]],words[index[1]], weight=sims[index[0], index[1]])

In [9]:
weight_values = nx.get_edge_attributes(G, 'weight')
weight_values[('egg', 'eggs')]

0.78060067

In [11]:
# nx.spring_layout?
positions = nx.spring_layout(G)

In [15]:
positions

{'egg': array([0.81344342, 0.43930036]),
 'eggs': array([0.82259965, 0.42835954]),
 'orange': array([ 0.67533004, -0.43486479]),
 'lemon': array([ 0.68768239, -0.43293366]),
 'lime': array([ 0.68482649, -0.42228186]),
 'thyme': array([-0.34362325,  0.24652377]),
 'parsley': array([-0.35302714,  0.21413659]),
 'cilantro': array([-0.358695  ,  0.15697496]),
 'tomato': array([-0.72574949, -0.55222344]),
 'tomatoes': array([-0.69078487, -0.52389491]),
 'clove': array([-0.55851573, -0.56973767]),
 'cloves': array([-0.58405203, -0.59687424]),
 'stock': array([-0.8111195 , -0.26325268]),
 'broth': array([-0.82588398, -0.26278967]),
 'leaf': array([-0.8568368 ,  0.41397005]),
 'leaves': array([-0.87678331,  0.42479923]),
 'pork': array([-0.39448065,  0.09866139]),
 'beef': array([-0.40612596,  0.11094632]),
 'oregano': array([-0.35855207,  0.27251333]),
 'chile': array([ 0.63179576, -0.22349751]),
 'chili': array([ 0.66438395, -0.21304305]),
 'canola': array([-0.35041952,  0.64190614]),
 'vege

In [16]:
nx.set_node_attributes(G, name='position', values=positions)

In [21]:
searches = ['potato', 'gouda', 'crab']

In [22]:
edge_x = []
edge_y = []
weights = []
ave_x, ave_y = [], []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['position']
    x1, y1 = G.nodes[edge[1]]['position']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    ave_x.append(np.mean([x0, x1]))
    ave_y.append(np.mean([y0, y1]))
    weights.append(f'{edge[0]}, {edge[1]}: {weight_values[(edge[0], edge[1])]}')

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    opacity=0.7,
    line=dict(width=2, color='White'),
    hoverinfo='text',
    mode='lines')

edge_trace.text = weights


node_x = []
node_y = []
sizes = []
for node in G.nodes():
    x, y = G.nodes[node]['position']
    node_x.append(x)
    node_y.append(y)
    if node in searches:
        sizes.append(50)
    else:
        sizes.append(15)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        line=dict(color='White'),
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Picnic',
        reversescale=False,
        color=[],
        opacity=0.9,
        size=sizes,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2
    )
)

invisible_similarity_trace = go.Scatter(
    x=ave_x, y=ave_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=[],
        opacity=0,
    )
)

invisible_similarity_trace.text=weights

In [23]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(adjacencies[0])

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [24]:
fig = go.Figure(
    data=[edge_trace, node_trace, invisible_similarity_trace],
    layout=go.Layout(
        title='Network Graph of Word Embeddings',
        template='plotly_dark',
        titlefont_size=20,
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        annotations=[
            dict(
                text="Adapted from: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.005, y=-0.002 ) 
        ],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
)

plot(fig)

'temp-plot.html'